In [ ]:
#Import external packages
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats
import pandas as pd
%matplotlib inline

#Import internal packages
import evolve
import plot
import fromexo
import getall as get
from mineralDB import minerals
from constants import *

# Convenience functions
Pe = lambda n: format(n, '.4e')
Pf = lambda n: format(n, '.4f')

In [ ]:
# Should I import planetary params from ExoPlex? If not, my_composition is assumed.
ExoPlex='FALSE'
method='dynamic'    # dynamic, benchmark, MC, or default parameters? assumes default values for those not provided explicitly 
ID = method

# Where should I send outputs from this run?
outfolder = 'OUTPUT/'
outfile = 'earth_nomantleFe.csv'

# What exoplex file should I import, if there is one, and on what line does the core stop and mantle start?
file='earth_nomantleFe_FeMg0.9_0.07_0.9_0.09_0.9.csv'
startline=1000

In [ ]:
# Info about your planet - note that Mpl and Rpl are *overwritten* if ExoPlex='TRUE'
Tp0=2000.           # starting mantle potential temperature in K        Earth = 1800.0 (initial), 1650 (present)
Mpl=1.0             # Planet mass in Me - usually between 0.5 and 5. ignored if ExoPlex = 'TRUE' Earth = 1.0
Rpl=1.0             # Relative heat production per kg mantle, vs Earth  Earth = 1.0
Qpl=1.0             # Planet's starting radiogenic abundance, per kg mantle
Pref=30.0            # reference pressure for thermal calculations, in GPa. If Pref is less than 4, Pref is set to half the CMB pressure.
tmax=4.55         # ending time, in Ga - how long to cool the planet         Earth = 4.55
my_composition = {'O': 1.0}
keepconstant = {'beta': 0.33}

planet={'ID': ID, 'Mpl':Mpl, 'Rpl':Rpl, 'Qpl':Qpl, 'Tp0':Tp0, 'Pref':Pref, 
     'outcols': ['ID', 'time', 'temp', 'Ra', 'H', 'Q', 'Urey', 'viscT', 
     'visc0', 'Ev', 'log10visc', 'beta'], 'constants': keepconstant, 'method': method,
        'Water': None}
planet, thermals = get.setup(planet, ExoPlex, file, startline, my_composition)
if ExoPlex=='FALSE':
    planet.update(DEFAULT)
if method=='dynamic':
    thermals=get.thermals_at_P_ave(planet['composition'], Pref)


In [ ]:
# Evolve your planet over time.
Evolution = evolve.ThermEv(planet, thermals, method, planet['Tp0'], tmax)
p = plot.evolution_colorcoded(Evolution, 'viscT', 'continuous')

In [ ]:
pd.DataFrame(thermals, columns=['temp', 'alpha', 'Cp', 'k']).plot(x='temp', y='alpha')
pd.DataFrame(thermals, columns=['temp', 'alpha', 'Cp', 'k']).plot(x='temp', y='Cp')
pd.DataFrame(thermals, columns=['temp', 'alpha', 'Cp', 'k']).plot(x='temp', y='k')